In [1]:
import random
import string
import time

# 1. Przyjmij następujący zbiór danych wejściowych:

In [2]:
data1 = "bbb$"
data2 = "aabbabd"
data3 = "ababcd"
data4 = "abaababaabaabaabab$"
data5 = ''.join(random.choices(string.ascii_lowercase, k=100))
data6 = None

with open('1997_714_head.txt', 'r') as file:
    data6 = file.read()

# 2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

In [16]:
def marker_check(text, marker='$'):
    lines = text.splitlines()
    processed = []

    for line in lines:
        if not line.endswith(marker):
            line += marker
        processed.append(line)

    return '\n'.join(processed)


test_data = "abc"
print("Before: " + test_data)
processed_data = marker_check(test_data)
print("After: " + processed_data)
print("-------------------")

test_data = "abcd$"
print("Before: " + test_data)
processed_data = marker_check(test_data)
print("After: " + processed_data)
print("-------------------")


Before: abc
After: abc$
-------------------
Before: abcd$
After: abcd$
-------------------


# 3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [4]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self, text):
        self.root = TrieNode()
        self.construct_(text)

    def construct_(self, text):
        for i in range(len(text)):
            current_node = self.root
            for j in range(i, len(text)):
                char = text[j]
                if char not in current_node.children:
                    current_node.children[char] = TrieNode()
                current_node = current_node.children[char]
            current_node.is_end_of_word = True

    def search_(self, pattern):
        current_node = self.root
        for char in pattern:
            if char not in current_node.children:
                return False
            current_node = current_node.children[char]
        return True

# 4. Zaimplementuj algorytm konstruujący drzewo sufiksów.

In [5]:
class SuffixTreeNode:
    def __init__(self, char):
        self.char = char
        self.children = {}

class SuffixTree:
    def __init__(self, text):
        self.root = SuffixTreeNode('')
        self.construct_(text)
        
    def insert_(self, suffix):
        current_node = self.root
        for char in suffix:
            if char not in current_node.children:
                current_node.children[char] = SuffixTreeNode(char)
            current_node = current_node.children[char]

    def construct_(self, text):
        n = len(text)
        for i in range(n):
            self.insert_(text[i:])

    def search_(self, pattern):
        current_node = self.root
        for char in pattern:
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                return False
        return True


# 5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.

Algorytm wyszukiwania wzorców (Algorytm Knutha-Morrisa-Pratta) został wzięty z laboratorium pierwszego.

In [6]:
def prefix_function(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0

    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k

    return pi


def KMP(text, pattern):
    n = len(text)
    m = len(pattern)
    pi = prefix_function(pattern)
    q = 0
    result = []

    for i in range(n):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == m:
            result.append(i - m + 1)
            q = pi[q-1]

    return result

Funkcja testująca:

In [17]:
def test_(text, pattern):
    trie = Trie(text)
    trie_occurrence = trie.search_(pattern)
    suffix = SuffixTree(text)
    suffix_occurrence = suffix.search_(pattern)
    occ = KMP(text, pattern)
    kmp_occurrence = len(occ) > 0
    if kmp_occurrence and trie_occurrence and suffix_occurrence:
        print("Znaleziono wzorzec w każdej strukturze.")
    elif kmp_occurrence and trie_occurrence:
        print("Nie znaleziono wzorca w Suffix Tree.")
    elif kmp_occurrence and suffix_occurrence:
        print("Nie znaleziono wzorca w Trie.")
    elif suffix_occurrence and kmp_occurrence:
        print("Nie znaleziono wzorca w KMP.")
    else:
        print("Nie znaleziono wzorca nigdzie.")

        
test_("bananasbananas", "ananas")
test_("aaaaaaaaaa", "a")
test_("bananasbananas", "banana")
test_("asd","sd")

Znaleziono wzorzec w każdej strukturze.
Znaleziono wzorzec w każdej strukturze.
Znaleziono wzorzec w każdej strukturze.
Znaleziono wzorzec w każdej strukturze.


# 6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
<ul>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (1p.),</li>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (1p.),</li>
    <li>Drzewo sufiksów - algorytm Ukkonena (3p).</li>
    </ul>

In [9]:
def measure_time(data):
    start_time = time.time()
    trie = Trie(data)
    time_trie = time.time() - start_time

    start_time = time.time()
    suffix_tree = SuffixTree(data)
    time_suffix_tree = time.time() - start_time

    print(f"Czas konstrukcji Trie: {time_trie} s")
    print(f"Czas konstrukcji Suffix Tree dla: {time_suffix_tree} s")
    
print("Tests for data1")
print("----------------------")
measure_time(data1)
print("Tests for data2")
print("----------------------")
measure_time(data2)
print("Tests for data3")
print("----------------------")
measure_time(data3)
print("Tests for data4")
print("----------------------")
measure_time(data4)
print("Tests for data5")
print("----------------------")
measure_time(data5)
print("Tests for data6")
print("----------------------")
measure_time(data6)

Tests for data1
----------------------
Czas konstrukcji Trie: 1.5735626220703125e-05 s
Czas konstrukcji Suffix Tree dla: 1.3113021850585938e-05 s
Tests for data2
----------------------
Czas konstrukcji Trie: 2.288818359375e-05 s
Czas konstrukcji Suffix Tree dla: 1.9788742065429688e-05 s
Tests for data3
----------------------
Czas konstrukcji Trie: 5.1975250244140625e-05 s
Czas konstrukcji Suffix Tree dla: 2.7179718017578125e-05 s
Tests for data4
----------------------
Czas konstrukcji Trie: 0.00010013580322265625 s
Czas konstrukcji Suffix Tree dla: 0.00013709068298339844 s
Tests for data5
----------------------
Czas konstrukcji Trie: 0.005153179168701172 s
Czas konstrukcji Suffix Tree dla: 0.004499197006225586 s
Tests for data6
----------------------
Czas konstrukcji Trie: 8.083142757415771 s
Czas konstrukcji Suffix Tree dla: 8.516007900238037 s
